## HW 4 Probabilistic Models
## Joby George (jg6615)

## Due 3/23/2022





# Problem 1 Prompt:

Prove equivalence of ERM for binary classification using logistic loss and probabilistic approach to logistic regression results in the same minimization problem

## Problem 1 Answer

### MLE Expression for logistic Regression

Using slides 45 and 46 from week two of lecture, we know the logistic log function is 
$$ L(\theta) = \frac{1}{n}\sum_i^{n}log(1 +e^{-m}) \space and $$

where n = the number of data points

$$ m =  y_i\theta^{T} x_i $$

Which is equivalent to:

$$ L(\theta) = \frac{1}{n}\sum_i^{n}log(1 +e^{-y_i\theta^{T}x_i}) \space \text  where \space y \space \in {-1,1}$$

Which is the expression we would like to minimize to minimize empirical risk.

### Negative Log-likelihood for logistic regression

### Case 1 Y_i = 0, (-1 in MLE Outcome Space) 

From slide 19 we observe the log-likelihood we would like to maximize:

$$NLL = -\sum_{i}^{n}y_{i} log(f(\theta^{T}x_{i}))+(1-y_{i})(log(1−f(\theta^{T}x_{i})) \space where \space y_{i} \space \in {0,1} \space and$$

$$f(\eta) = \frac{1}{1+e^{\eta}}$$

if $y_{i}$ = 0, we observe this to be:

$$NLL = \sum_{i}^{n}-log(1-f(\theta^{T}x_{i})) \space $$


plugging  in our logistic link function, if $y_{i}$ = 0 we get:

$$NLL = \sum_{i}^{n}-log(1-\frac{1}{1+e^{(-\theta^{T}x_{i})}}) \space $$

setting a common denominator and expanding our logistic, we get: 
$$NLL = \sum_{i}^{n}-log(\frac{1+e^{(-\theta^{T}x_{i})}-1}{1+e^{(-\theta^{T}x_{i})}}) \space $$

simplifying this we get 
$$NLL = \sum_{i}^{n}-log(\frac{e^{(-\theta^{T}x_{i})}}{1+e^{(-\theta^{T}x_{i})}}) \space $$

multiplying the numerator and denominator by one, expressed as:
$$\frac{e^{(\theta^{T}x_{i})}}{e^{(\theta^{T}x_{i})}}$$

we get:

$$NLL = \sum_{i}^{n}-log(\frac{e^{0}}{(1+e^{(-\theta^{T}x_{i})})   (e^{(\theta^{T}x_{i})})}) \space $$


which simplifies to 
$$NLL = \sum_{i}^{n}-log(\frac{1}{1+e^{(\theta^{T}x_{i})}}) \space $$

if we apply the negative sign to the log by taking the reciprocal of it's input, and scale it by a factor of $\frac{1}{n}$ we end with our final expression:

$$NLL = \frac{1}{n}\sum_{i}^{n}log(1+e^{\theta^{T}x_{i}})$$

This is equivalent to the MLE when $y_{i}$ = -1:

$$ L(\theta) = \frac{1}{n}\sum_i^{n}log(1 +e^{\theta^{T}x_i}) $$

### Case 1 Y_i = 1, (1 in MLE Outcome Space) 


$$NLL = \sum_{i}^{n}-log(f(\theta^{T}x_{i})) \space $$

plugging  in our logistic link function, if $y_{i}$ = 1 we get:

$$NLL = \sum_{i}^{n}-log(\frac{1}{1+e^{(-\theta^{T}x_{i})}}) \space $$

if we apply the negative sign to the log by taking the reciprocal of it's input, and scale it by a factor of $\frac{1}{n}$ we end with our final expression:

$$NLL = \frac{1}{n} \sum_{i}^{n}log(1+e^{(-\theta^{T}x_{i})}) \space $$

This is equivalent to the MLE when $y_{i}$ = 1:

$$ L(\theta) = \frac{1}{n}\sum_i^{n}log(1 +e^{-\theta^{T}x_i}) $$

### Q.E.D